# import Data

I had taken refrence from Topic Modeling and Latent Dirichlet Allocation (LDA) in Python from toward data science article by Susan Li

In [1]:
import pandas as pd
data=pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data.head(5)

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [2]:
data_text=data[['headline_text']]
data_text['index']=data_text.index
text=data_text
print(len(text))
text.head(5)

1103663


,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


# preprocess Data

In [3]:
import gensim
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np
np.random.seed(2018)
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')

C:\Users\aiman\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aiman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
def lemmatize(text):
    return PorterStemmer().stem(WordNetLemmatizer().lemmatize(text,'v'))
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in STOPWORDS and len(token)>3:
            result.append(lemmatize(token))
    return result

In [5]:
print(text['headline_text'].isnull().sum())
doc_sample = text[text['index'] == 4310].astype(str).values[0][0]
words = doc_sample.split(' ')
print("before preprocessing\n",words)
p=preprocess(doc_sample)
print(p,'hhh')

0
before preprocessing
 ['rain', 'helps', 'dampen', 'bushfires']
['rain', 'help', 'dampen', 'bushfir'] hhh


In [6]:
doc_sample

'rain helps dampen bushfires'

In [7]:
text_pre = text['headline_text'].map(preprocess)
text_pre.head(5)

0        [decid, commun, broadcast, licenc]
1                        [wit, awar, defam]
2    [call, infrastructur, protect, summit]
3               [staff, aust, strike, rise]
4      [strike, affect, australian, travel]
Name: headline_text, dtype: object

# Bag of words

In [8]:
#the number of times a word appears in the training set.
dictionary = gensim.corpora.Dictionary(text_pre)

In [9]:
#Filter out tokens that appear in dictonary
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [10]:
#For each document we create a dictionary reporting how many
#words and how many times those words appear
bow_corpus = [dictionary.doc2bow(doc) for doc in text_pre]

# LDA Bag of Words model

In [11]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [12]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.024*"world" + 0.022*"adelaid" + 0.018*"market" + 0.014*"power" + 0.013*"record" + 0.013*"share" + 0.012*"trial" + 0.012*"leagu" + 0.011*"fall" + 0.011*"tasmania"
Topic: 1 
Words: 0.021*"say" + 0.019*"north" + 0.016*"elect" + 0.012*"labor" + 0.011*"minist" + 0.010*"victoria" + 0.010*"health" + 0.009*"state" + 0.009*"talk" + 0.009*"budget"
Topic: 2 
Words: 0.029*"charg" + 0.028*"court" + 0.021*"murder" + 0.019*"polic" + 0.018*"face" + 0.017*"year" + 0.015*"jail" + 0.014*"accus" + 0.013*"drug" + 0.013*"child"
Topic: 3 
Words: 0.017*"donald" + 0.015*"turnbul" + 0.014*"deal" + 0.012*"say" + 0.012*"famili" + 0.011*"centr" + 0.011*"christma" + 0.010*"drum" + 0.009*"research" + 0.009*"island"
Topic: 4 
Words: 0.038*"polic" + 0.022*"coast" + 0.019*"interview" + 0.018*"miss" + 0.018*"perth" + 0.018*"shoot" + 0.015*"investig" + 0.014*"gold" + 0.013*"death" + 0.011*"victim"
Topic: 5 
Words: 0.026*"kill" + 0.021*"die" + 0.021*"countri" + 0.020*"live" + 0.016*"crash" + 0.014*"dead

# Testing model on unseen document

In [13]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.34999993443489075	 Topic: 0.021*"say" + 0.019*"north" + 0.016*"elect" + 0.012*"labor" + 0.011*"minist"
Score: 0.1833333522081375	 Topic: 0.017*"donald" + 0.015*"turnbul" + 0.014*"deal" + 0.012*"say" + 0.012*"famili"
Score: 0.18333333730697632	 Topic: 0.024*"world" + 0.022*"adelaid" + 0.018*"market" + 0.014*"power" + 0.013*"record"
Score: 0.18333333730697632	 Topic: 0.038*"polic" + 0.022*"coast" + 0.019*"interview" + 0.018*"miss" + 0.018*"perth"
Score: 0.01666666753590107	 Topic: 0.029*"charg" + 0.028*"court" + 0.021*"murder" + 0.019*"polic" + 0.018*"face"
Score: 0.01666666753590107	 Topic: 0.026*"kill" + 0.021*"die" + 0.021*"countri" + 0.020*"live" + 0.016*"crash"
Score: 0.01666666753590107	 Topic: 0.035*"trump" + 0.023*"south" + 0.021*"hous" + 0.017*"brisban" + 0.016*"plan"
Score: 0.01666666753590107	 Topic: 0.021*"govern" + 0.014*"rural" + 0.013*"indigen" + 0.012*"work" + 0.012*"fund"
Score: 0.01666666753590107	 Topic: 0.033*"sydney" + 0.022*"australia" + 0.017*"final" + 0.0